<a href="https://colab.research.google.com/github/SushantVij/ML_WeatherPrediction/blob/main/seattle_weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WEATHER PREDICTION

Install and Import Dependencies

In [ ]:
!pip install stats

Read in Data and Process Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re
import missingno as mso
import stats
from scipy import stats
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
from google.colab import files
from IPython.display import Image

In [ ]:
uploaded=files.upload()

In [ ]:
data=pd.read_csv("/content/seattle-weather.csv")
data.head()

In [ ]:
data.shape

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Function to ignore all warnings by setting 'ignore' as a parameter

# Use sns.countplot() to visualize the weather data
sns.countplot(x="weather", data=data, palette='hls')
plt.show()  # Don't forget to show the plot using plt.show()


In [ ]:
countrain=len(data[data.weather=='rain'])
print(countrain)
percentage=(countrain/len(data.weather))*100
print(percentage)

In [ ]:
countsun=len(data[data.weather=='sun'])
print(countsun)
percentage=(countsun/len(data.weather))*100
print(percentage)

In [ ]:
countdrizzle=len(data[data.weather=='drizzle'])
print(countdrizzle)
percentage=(countdrizzle/len(data.weather))*100
print(percentage)

In [ ]:
countsnow=len(data[data.weather=='snow'])
print(countsnow)
percentage=(countsnow/len(data.weather))*100
print(percentage)

In [ ]:
countfog=len(data[data.weather=='fog'])
print(countfog)
percentage=(countfog/len(data.weather))*100
print(percentage)

In [ ]:
data[['precipitation','temp_max','temp_min','wind']].describe()
#view some basic statistical details like percentile, mean, std etc. of a data frame or a series of numeric values.

In [ ]:
sns.set(style='darkgrid')#seems to be changing the background color of the plot.
fig,axs=plt.subplots(2,2,figsize=(10,8))
#kde:bool(kerner density esitmate)polt is am ethod for visulization the distribuition of dataset,analogous to histogram
#If True, compute a kernel density estimate to smooth the distribution and show on the plot as (one or more) line(s).
#Only relevant with univariate data(simplest form of data for statistical analysis).
sns.histplot(data=data,x='precipitation',kde=True,ax=axs[0,0],color='green')#+skewed
sns.histplot(data=data,x='temp_max',kde=True,ax=axs[0,1],color='red')#-skewed+outliers
sns.histplot(data=data,x='temp_min',kde=True,ax=axs[1,0],color='blue')#-skewed+outliers
sns.histplot(data=data,x='wind',kde=True,ax=axs[1,1],color='orange')#+skewed

In [ ]:
print("skewness: {:.3f}".format(data['precipitation'].skew()))
print("kurtosis: {:.3f}".format(data['precipitation'].kurt()))

In [ ]:
print("skewness: {:.3f}".format(data['temp_max'].skew()))
print("kurtosis: {:.3f}".format(data['temp_max'].kurt()))

In [ ]:
print("skewness: {:.3f}".format(data['temp_min'].skew()))
print("kurtosis: {:.3f}".format(data['temp_min'].kurt()))

In [ ]:
print("skewness: {:.3f}".format(data['wind'].skew()))
print("kurtosis: {:.3f}".format(data['wind'].kurt()))

In [ ]:
#to understand skewness of data
# A violin plot is a cross between a box plot and a kernel density plot that displays data peaks.
#It’s used to show how numerical data is distributed.
sns.set(style='darkgrid')
fig,axs=plt.subplots(2,2,figsize=(10,8))
sns.violinplot(data=data,x='precipitation',kde=True,ax=axs[0,0],color='green')
sns.violinplot(data=data,x='temp_max',kde=True,ax=axs[0,1],color='red')
sns.violinplot(data=data,x='temp_min',kde=True,ax=axs[1,0],color='blue')
sns.violinplot(data=data,x='wind',kde=True,ax=axs[1,1],color='orange')

In [ ]:
# Boxplot is also used to detect outliers in the data set. It captures the summary of the data efficiently with a simple box and whiskers
# and allows us to compare easily across groups
plt.figure(figsize=(12, 6))
sns.boxplot(x='weather', y='precipitation', data=data, palette='hls')
plt.show()  # Don't forget to show the plot using plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='weather', y='temp_max', data=data, palette='inferno')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='weather', y='wind', data=data, palette='hls')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='weather', y='temp_min', data=data, palette='inferno')
plt.show()


In [ ]:
#Heatmap is a visualization that displays data in a color encoded matrix.
#The intensity of color varies based on the value of the attribute represented in the visualization.
#+ve corelation b/w min and max temp
#+ve corelation b/w precipitation b/w wind
plt.figure(figsize=(12,6))
sns.heatmap(data.corr(),annot=True,cmap='coolwarm')

Preprocessing and Cleaning

In [ ]:
data.isna().sum()

In [ ]:
plt.figure(figsize=(12,6))
axz=plt.subplot(1,2,2)
#function takes three arguments that describes the layout of the figure.
#The layout is organized in rows and columns, which are represented by the first and second argument.
#The third argument represents the index of the current plot.
#the figure has 1 row, 2 columns, and this plot is the second plot.
#Missingno library offers a very nice way to visualize the distribution of NaN values.
# Missingno is a Python library and compatible with Pandas.
#This bar chart gives you an idea about how many missing values are there in each column.
mso.bar(data.drop(['date'],axis=1),ax=axz,fontsize=12)

In [ ]:
data.dtypes

In [ ]:
data=data.drop(['date'],axis=1)

Train Model

In [ ]:
#dataframe.quantile() function return values at the given quantile over requested axis, a numpy.percentile.
Q1=data.quantile(0.25)
Q3=data.quantile(0.75)
IQR=Q3-Q1
data=data[~((data<(Q1-1.5*IQR))|(data>(Q3+1.5*IQR))).any(axis=1)]


In [ ]:
import numpy as np
#function is used to determine the positive square-root of an array, element-wise.
data.precipitation=np.sqrt(data.precipitation)
data.wind=np.sqrt(data.wind)

In [ ]:
sns.set(style='darkgrid')
fig,axs=plt.subplots(2,2,figsize=(10,8))
sns.histplot(data=data,x='precipitation',kde=True,ax=axs[0,0],color='green')
sns.histplot(data=data,x='temp_max',kde=True,ax=axs[0,1],color='red')
sns.histplot(data=data,x='temp_min',kde=True,ax=axs[1,0],color='blue')
sns.histplot(data=data,x='wind',kde=True,ax=axs[1,1],color='orange')

In [ ]:
data.head()

In [ ]:
lc=LabelEncoder()
data['weather']=lc.fit_transform(data['weather'])# a fit_transform() to the training data. The model we created, in this case,
#will discover the mean and variance of the characteristics in the training set.

In [ ]:
x=((data.loc[:,data.columns!='weather']).astype(int)).values[:,0:]
y=data['weather'].values


In [ ]:
data.weather.unique()

ACCURACY

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=2)

In [ ]:

print('There are {} samples in the training set and {} samples in the test set'.format(
x_train.shape[0], x_test.shape[0]))
print()
#For many machine learning algorithms, it is important to scale the data. Let's do that now using sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)
X_train_std = sc.transform(x_train)
X_test_std = sc.transform(x_test)




In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
print('KNN Aaccuracy:{:.2f}%'.format(knn.score(x_test,y_test)*100))
print('The accuracy of the knn classifier is {:.2f} out of 1 on training data'.format(knn.score(X_train_std, y_train)))
print('The accuracy of the knn classifier is {:.2f} out of 1 on test data'.format(knn.score(X_test_std, y_test)))

In [ ]:
svm=SVC()
svm.fit(x_train,y_train)
print('SVM Aaccuracy:{:.2f}%'.format(svm.score(x_test,y_test)*100))
print('The accuracy of the svm classifier on training data is {:.2f} out of 1'.format(svm.score(X_train_std, y_train)))
print('The accuracy of the svm classifier on test data is {:.2f} out of 1'.format(svm.score(X_test_std, y_test)))

In [ ]:
gbc=GradientBoostingClassifier(subsample=0.5,n_estimators=450,max_depth=5,max_leaf_nodes=25)
gbc.fit(x_train,y_train)
print('GBC Aaccuracy:{:.2f}%'.format(gbc.score(x_test,y_test)*100))


In [ ]:
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
xgb=XGBClassifier()

xgb_clf = xgb.fit(X_train_std, y_train)
xgb.fit(x_train,y_train)
print('XGB Aaccuracy:{:.2f}%'.format(xgb.score(x_test,y_test)*100))
print('The accuracy of the xgb classifier is {:.2f} out of 1 on training data'.format(xgb_clf.score(X_train_std, y_train)))
print('The accuracy of the xgb classifier is {:.2f} out of 1 on test data'.format(xgb_clf.score(X_test_std, y_test)))

Forecast Away

In [ ]:
import numpy as np

input = np.array([[0, 25, 19, 4.56774]])
ot = xgb.predict(input)
print('The weather is: ')
if ot == 0:
    print('Drizzle')
elif ot == 1:
    print('Fog')
elif ot == 2:
    print('Rain')
elif ot == 3:
    print('Snow')
else:
    print('Sun')
